# Sequential model

In [ ]:
import os
from pathlib import Path
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [ ]:
import pandas as pd
import numpy as np

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, array_to_img, load_img

import matplotlib.pyplot as plt

from sklearn.datasets import load_files

In [ ]:
from google.colab import files

files.upload(); 

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir /root/.kaggle/
!mv kaggle.json /root/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v /content

- path is now set to: /content


In [ ]:
!kaggle datasets download lexset/synthetic-asl-alphabet

100% 6.58G/6.58G [03:44<00:00, 32.3MB/s]
100% 6.58G/6.58G [03:44<00:00, 31.5MB/s]


In [ ]:
!unzip /content/datasets/lexset/synthetic-asl-alphabet/synthetic-asl-alphabet.zip

In [ ]:
train_dir = "/content/Train_Alphabet"
test_dir = "/content/Test_Alphabet"

In [ ]:
image_gen = ImageDataGenerator(validation_split=0.3, rotation_range=20, brightness_range=(0.6, 1.0), shear_range=15.0)

In [ ]:
dim_x, dim_y = 512,512
batch_size=32
input_shape = (dim_x,dim_y,3)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(27, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(27))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(27))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
train_generator = image_gen.flow_from_directory(
    train_dir, target_size=(512,512),batch_size=batch_size, class_mode='categorical', subset='training')

val_generator = image_gen.flow_from_directory(
    train_dir, target_size=(512,512),batch_size=batch_size, class_mode='categorical', subset='validation')

test_generator = image_gen.flow_from_directory(
    test_dir, target_size=(512,512),batch_size=batch_size, class_mode='categorical')

Found 17010 images belonging to 27 classes.
Found 7290 images belonging to 27 classes.
Found 2700 images belonging to 27 classes.


In [ ]:
print(train_generator.class_indices)

{'A': 0, 'B': 1, 'Blank': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}


In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=433,
        epochs=7,
        validation_data=val_generator,
        validation_steps=100)

In [ ]:
results = model.evaluate(test_generator)
print(results)

In [ ]:
!mkdir -p saved_model
model.save('saved_model/sequential2') 
new_model = tf.keras.models.load_model('saved_model/sequential2')

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 510, 510, 32)      896       
                                                                 
 activation_15 (Activation)  (None, 510, 510, 32)      0         
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 255, 255, 32)     0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 253, 253, 32)      9248      
                                                                 
 activation_16 (Activation)  (None, 253, 253, 32)      0         
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 126, 126, 32)     0         
 g2D)                                                 

In [ ]:
export_path = 'export_model2/'
tf.saved_model.save(model, os.path.join('/content/',export_path))

download_path = '/content/seq_model2.zip'
!zip -r {download_path} {export_path}
files.download(download_path)

  adding: export_model2/ (stored 0%)
  adding: export_model2/assets/ (stored 0%)
  adding: export_model2/saved_model.pb (deflated 90%)
  adding: export_model2/variables/ (stored 0%)
  adding: export_model2/variables/variables.index (deflated 66%)
  adding: export_model2/variables/variables.data-00000-of-00001 (deflated 8%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>